<a href="https://colab.research.google.com/github/msalman-abid/CUDA_Spring2022_GoogleColabs/blob/main/smartcrop_cuda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%cd /usr/local/
!rm -rf cuda
!ln -s /usr/local/cuda-10.0 /usr/local/cuda
!stat cuda
%cd /content

/usr/local
  File: cuda -> /usr/local/cuda-10.0
  Size: 20        	Blocks: 0          IO Block: 4096   symbolic link
Device: 24h/36d	Inode: 2632276     Links: 1
Access: (0777/lrwxrwxrwx)  Uid: (    0/    root)   Gid: (    0/    root)
Access: 2022-04-08 04:29:57.363457553 +0000
Modify: 2022-04-08 04:29:57.253458415 +0000
Change: 2022-04-08 04:29:57.253458415 +0000
 Birth: -
/content


In [2]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Sat_Aug_25_21:08:01_CDT_2018
Cuda compilation tools, release 10.0, V10.0.130


In [25]:
from skimage import io
from PIL import Image
image = Image.fromarray(io.imread("https://placekitten.com/800/571"))
image.save("kitten.png")

In [35]:
%%writefile smartcrop.cu
#include <stdio.h>
#include <time.h>

// OpenCV Imports
#include <opencv2/imgproc.hpp>
#include <opencv2/imgcodecs.hpp>
#include <opencv2/highgui.hpp>

#define ARR_SIZE 1024
#define BLK_SIZE 1024 // threads in each block

#define WIDTH = 512;
#define HEIGHT = 512;
#define SIZE = WIDTH*HEIGHT;
#define IMAGE_SIZE_IN_BYTES = SIZE * sizeof(unsigned int) * 4; // 4 channel image of HxW dimensions

inline cudaError_t checkCudaErr(cudaError_t err, const char* msg) {
  if (err != cudaSuccess) {
    fprintf(stderr, "CUDA Runtime error at %s: %s\n", msg, cudaGetErrorString(err));
  }
  return err;
}

//void edgeDetectHost(int* i_data, int* o_data)

using namespace cv;

int main(int argc, char** argv)
{
  
    Mat src, src_gray, dst;
    int kernel_size = 3;
    int scale = 1;
    int delta = 0;
    int ddepth = CV_16S;
    
    const char* imageName = argc >=2 ? argv[1] : "kitten.png";
    

    src = imread( imageName, IMREAD_COLOR ); // Load an image
    
    // Check if image is loaded fine
    if(src.empty()){
        printf(" Error opening image\n");
        return -1;
    }
 
    // Reduce noise by blurring with a Gaussian filter ( kernel size = 3 )
    GaussianBlur( src, src, Size(3, 3), 0, 0, BORDER_DEFAULT );
    cvtColor( src, src_gray, COLOR_BGR2GRAY ); // Convert the image to grayscale
    
    Mat abs_dst;
    Laplacian( src_gray, dst, ddepth, kernel_size, scale, delta, BORDER_DEFAULT );
    
    // converting back to CV_8U
    convertScaleAbs( dst, abs_dst );
    
    imwrite("kitten_laplace.png", abs_dst);


}

Overwriting smartcrop.cu


In [37]:
!nvcc /content/smartcrop.cu -o smartcrop `pkg-config --cflags --libs opencv`

In [38]:
!./smartcrop